In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import talib

# Load data


In [46]:
PATH = "../Data/DAX/yahoo_fin/"

def load(interval="1d"):
     data = pd.read_csv(PATH+"{}.csv".format(interval))
     return data
def save(filename, data):
     data.to_excel(PATH+filename+".xlsx")

data = load()
period = period[:200]
tprices = talib.TYPPRICE(period['high'], period['low'], period['close'])
fig = go.Figure(data=[go.Candlestick(x=period[period.columns[0]],
                open=period['open'], high=period['high'],
                low=period['low'], close=period['close'])
                     ])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

# Clean the data

Get rid of None

@1030
@2564 (volume)


In [57]:
def clean(data):
    id1 = id2 = 0

    for i in range(len(data["close"])):
        if (data["close"][i] != data["open"][i]):
            id1 = i
            break

    for i in range(len(data["close"])):
        if (data["volume"][i] > 0 ):
            id2 = i
            break

    if (id1 > 0):
        samples1 = data.drop(np.arange(id1))
    else:
        samples1 = data
    
    if (id2 > 0):
        samples2 = data.drop(np.arange(id2))
    else:
        samples2 = data

    samples1 = samples1.drop(columns=[samples1.columns[0], "ticker", "adjclose"])
    samples2 = samples2.drop(columns=[samples2.columns[0], "ticker", "adjclose"])

    samples1.reset_index(drop=True, inplace=True)
    samples1 = samples1.drop(np.where(np.isnan(samples1["close"]))[0])
    samples2.reset_index(drop=True, inplace=True)
    samples2 = samples2.drop(np.where(np.isnan(samples2["close"]))[0])

    return samples1, samples2

dataD = load(interval="1d")
dataW = load(interval="1wk")
dataM = load(interval="1mo")

dataD, dataD_V = clean(dataD)
dataW, dataW_V = clean(dataW)
dataM, dataM_V = clean(dataM)

save("data_D", dataD)
save("data_DV", dataD_V)
save("data_W", dataW)
save("data_WV", dataW_V)
save("data_M", dataM)
save("data_MV", dataM_V)





# BBANDS


In [18]:
upper, middle, lower = bbands = talib.BBANDS(tprices)

fig.add_trace(go.Scatter(x=period[period.columns[0]], y=lower, name='Lower',
                         line = dict(color='black', width=3, dash='dash')))

fig.add_trace(go.Scatter(x=period[period.columns[0]], y=upper, name='Upper',
                         line = dict(color='royalblue', width=3, dash='dash'), fill='tonexty'))

fig.add_trace(go.Scatter(x=period[period.columns[0]], y=middle, name='Middle',
                         line = dict(color='yellow', width=3, dash='dash')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()
